In [8]:
from bs4 import BeautifulSoup
import urllib
import re
import json

In [9]:
transcript_links = ['pilot0.html', 'episode6.html', 'episode7.html']

In [10]:
wiki_info = urllib.urlopen('https://en.wikipedia.org/wiki/List_of_Twin_Peaks_episodes').read()
wiki_soup = BeautifulSoup(wiki_info, 'html.parser')

In [11]:
description_array=[]
for a in wiki_soup.find_all("td", "description"):
    description_array.append(a.get_text())
    
print description_array

[u"In the feature-length pilot episode, the small town of Twin Peaks, Washington, is shaken up when the body of high school student Laura Palmer is discovered near a riverbank, wrapped in plastic. FBI Agent Dale Cooper is called by local sheriff Harry Truman when Ronette Pulaski, a schoolmate of Laura, is found wandering on a bridge and then slips into a coma. Cooper sees a connection between Laura's death and the murder of another girl one year earlier. After finding a piece of paper under Laura's fingernail, similar to one he found in the fingernail of the girl who died earlier, Cooper suspects that same killer might have struck again. Meanwhile, Sheriff Truman arrests Laura's boyfriend, Bobby Briggs, who is secretly having an affair with a married woman named Shelly Johnson. Afterwards, Laura's mother has a terrifying nightmare.", u"Cooper's investigation into the murder of Laura Palmer continues, as her secret boyfriend James Hurley is interrogated, and it appears that both trucker

In [12]:
episode_title_soup = []
episode_title_array = []
for a in wiki_soup.find_all("td", "summary"):
    episode_title_soup.append(a.get_text())
for episode in episode_title_soup:
    title = re.search('\"(.*)\".*\n',episode).group(1)
    subtitle = re.search('\n\"(.*)\"', episode).group(1)
    episode_title_array.append({'episode_title': title, 'episode_subtitle':subtitle})

episode_title_array

[{'episode_subtitle': u'Northwest Passage', 'episode_title': u'Pilot'},
 {'episode_subtitle': u'Traces to Nowhere', 'episode_title': u'Episode 1'},
 {'episode_subtitle': u'Zen, or the Skill to Catch a Killer',
  'episode_title': u'Episode 2'},
 {'episode_subtitle': u'Rest in Pain', 'episode_title': u'Episode 3'},
 {'episode_subtitle': u'The One-Armed Man', 'episode_title': u'Episode 4'},
 {'episode_subtitle': u"Cooper's Dreams", 'episode_title': u'Episode 5'},
 {'episode_subtitle': u'Realization Time', 'episode_title': u'Episode 6'},
 {'episode_subtitle': u'The Last Evening', 'episode_title': u'Episode 7'},
 {'episode_subtitle': u'May the Giant Be with You',
  'episode_title': u'Episode 8'},
 {'episode_subtitle': u'Coma', 'episode_title': u'Episode 9'},
 {'episode_subtitle': u'The Man Behind Glass', 'episode_title': u'Episode 10'},
 {'episode_subtitle': u"Laura's Secret Diary", 'episode_title': u'Episode 11'},
 {'episode_subtitle': u"The Orchid's Curse", 'episode_title': u'Episode 12'}

In [13]:
count = 0
episode_metadata = episode_title_array
for num in description_array:
    
    episode_metadata[count]['episode_description'] = num
    count +=1

episode_metadata

[{'episode_description': u"In the feature-length pilot episode, the small town of Twin Peaks, Washington, is shaken up when the body of high school student Laura Palmer is discovered near a riverbank, wrapped in plastic. FBI Agent Dale Cooper is called by local sheriff Harry Truman when Ronette Pulaski, a schoolmate of Laura, is found wandering on a bridge and then slips into a coma. Cooper sees a connection between Laura's death and the murder of another girl one year earlier. After finding a piece of paper under Laura's fingernail, similar to one he found in the fingernail of the girl who died earlier, Cooper suspects that same killer might have struck again. Meanwhile, Sheriff Truman arrests Laura's boyfriend, Bobby Briggs, who is secretly having an affair with a married woman named Shelly Johnson. Afterwards, Laura's mother has a terrifying nightmare.",
  'episode_subtitle': u'Northwest Passage',
  'episode_title': u'Pilot'},
 {'episode_description': u"Cooper's investigation into t

In [14]:
def transcript_script(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, 'html.parser')
    script = soup.get_text()
    #gets all the scenes and characters
    scenes_chars = re.findall('\n\s*([A-Z]+[A-Z\s\.\-\:]*\s*)\n\s*', script)
    print scenes_chars
    scene = 0
    scene_array = []
    #parses the script into a dictionary
    for line in scenes_chars:
        #scene_num = re.search('(\d+\w*)[\.\: ]', line)
        #establishes the scene and all its info
        if 'INT' in line or 'EXT' in line:
            print line
            scene += 1
            scene_array.append({'scene_number': int(scene), 'characters':[], 'heading':{}})
            #inside or outside
            if "INT" in line:
                scene_array[scene-1]['heading']['int_ext'] = 'INT'
            elif "EXT" in line:
                scene_array[scene-1]['heading']['int_ext'] = 'EXT'
            #grabs location
            location = re.search('[A-Z]\.\s+(.*)\s+\-',line)
            if location is not None:
                scene_array[scene-1]['heading']['location'] = location.group(1)
            #gets time
            time = re.search('- ([A-Z]+)$',line)
            if time is not None:
                scene_array[scene-1]['heading']['time'] = time.group(1)
        elif scene == 0:
            pass
        else:
            if 'FADE' not in line and 'ACT' not in line and 'SCENE' not in line and 'CUT TO' not in line:
                character = re.search('\n*([A-Z .]+)\n*', line)
                if character is not None:
                    if character.group(1) not in scene_array[scene-1]['characters'] and line !=" ":
                        scene_array[scene-1]['characters'].append(character.group(1))
    print scene_array
    return scene_array

In [15]:
#This puts all of the data together and exports it to json
episode_array = []
for link in transcript_links:
    episode_num = int(re.search('(\d*).html',link).group(1))
    #this adds the episode metadata and scene information
    episode_array.append({'episode_number': episode_num, 'episode_description': episode_metadata[episode_num]['episode_description'], 'episode_title': episode_metadata[episode_num]['episode_title'], 'episode_subtitle': episode_metadata[episode_num]['episode_subtitle'], 'scenes': transcript_script(link)})
print episode_array
outfile = open('twin_peaks_transcripts.json', 'w')
json.dump(episode_array, outfile, indent=4)
outfile.close()

[u'FADE IN:\n', u'LOG LADY\n', u'FADE OUT:\n\n\n\n\nINTRO:\n\n\n\n\nFADE IN:\n', u'FADE OUT:\n\n\n\n\nFADE IN:\n\nEXT. LAKESHORE BY THE BLUE PINE LODGE - DAY\n', u'CUT TO:\n', u'INT. BLUE PINE LODGE - DAY\n', u'CUT TO:\n\nINT. BLUE PINE LODGE - DAY\n', u'PETE\n', u'CUT TO:\n\nINT. BLUE PINE LODGE - DAY\n', u'CUT TO:\n\nEXT. BLUE PINE LODGE - DAY\n', u'PETE\n', u'CUT TO:\n\nINT. BLUE PINE LODGE - DAY\n', u'PETE\n', u'CUT TO:\n', u'LUCY\n', u'SHERIFF TRUMAN\n', u'CUT TO:\n\nINT. BLUE PINE LODGE - DAY\n\nPETE\n', u'CUT TO:\n', u'SHERIFF TRUMAN\n', u'SHERIFF TRUMAN\n', u'LUCY\n', u'SHERIFF TRUMAN\n', u'SHERIFF TRUMAN\n', u'CUT TO:\n', u'CUT TO:\n\nEXT. LAKESHORE BY THE BLUE PINE LODGE - DAY\n', u'PETE\n', u'DR. HAYWARD\n', u'SHERIFF TRUMAN\n', u'DR. HAYWARD\n', u'ANDY\n', u'SHERIFF TRUMAN\n', u'ANDY\n', u'SHERIFF TRUMAN\n', u'ANDY\n', u'DR. HAYWARD\n', u'SHERIFF TRUMAN\n', u'DR. HAYWARD\n', u'ANDY\n', u'SHERIFF TRUMAN\n', u'ANDY\n', u'SHERIFF TRUMAN\n', u'ANDY\n', u'DR. HAYWARD\n', u'DR. H